In [ ]:
%loadFromPOM pom.xml

In [ ]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;
import org.hl7.fhir.instance.model.api.*;
import java.nio.file.*;

In [ ]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

背景：FHIR API

In [ ]:
// Load synthetic data from /synthetic-data/fhir directory
String path = "../synthetic-data/fhir";
try {
    Files.walk(Paths.get(path)).forEach(filePath -> {
        if (Files.isRegularFile(filePath)) {
            try {
                String resource = Files.readString(filePath);
                Bundle bundle = ctx.newJsonParser().parseResource(Bundle.class, resource);
                for (Bundle.BundleEntryComponent entry : bundle.getEntry()) {
                    Resource r = entry.getResource();
                    // Print resource type and id
                    String type = r.getResourceType().name();
                    if(type.equals("Patient") == true ){
                        // Print patient name
                        Patient patient = (Patient) r;
                        System.out.println(patient.getName().get(0).getGivenAsSingleString() + " " + patient.getName().get(0).getFamily());
                    }
                    //else if(type.equals("Organization") == true ){
                        // Print organization name
                    //    Organization organization = (Organization) r;
                    //    System.out.println(organization.getName());
                    //}
                }
                // Upload bundle to server
                // IBaseBundle outcome = client.transaction().withBundle(bundle).execute();

            } catch (Exception e) {
                System.out.println("Error: " + filePath + " " + e.getMessage());
            }
        }
    });
} catch (IOException e) {
    System.out.println("Error reading files: " + e.getMessage());
}


In [36]:
//read Organization "東部醫療網路" from DB
Bundle response = client
                .search()
                .forResource(Organization.class)
                .where(Organization.NAME.matchesExactly().value("東部醫療網路"))
                .returnBundle(Bundle.class)
                .execute();
Organization organization = (Organization) response.getEntry().get(0).getResource();

// Read all patient from DB
Bundle response = client.search().forResource(Patient.class).returnBundle(Bundle.class).execute();

// count patients
int count = 0;
while (true) {
    for (Bundle.BundleEntryComponent entry : response.getEntry()) {
        Resource resource = entry.getResource();
        if (resource instanceof Patient) {
            Patient patient = (Patient) resource;
            // Set general practitioner as "東部醫療網路"
            patient.setGeneralPractitioner(List.of(new Reference("Organization" + "/" + organization.getIdElement().getIdPart())));
            // Update patient
            MethodOutcome outcome = client.update().resource(patient).execute();
            //Print patient name & patient id
            System.out.println(patient.getName().get(0).getGivenAsSingleString() + " " + patient.getName().get(0).getFamily() + " " + patient.getId());
            count++;
        }
    }
    // Load next page
    if (response.getLink(Bundle.LINK_NEXT) != null) {
        response = client.loadPage().next(response).execute();
    } else {
        break;
    }
}
System.out.println("Total patients: " + count);

鴻裕 陳 http://localhost:8080/fhir/Patient/9869/_history/2
鴻 陳 http://localhost:8080/fhir/Patient/9952/_history/2
Christopher C.H. Parks http://localhost:8080/fhir/Patient/10002/_history/1
Christopher C.H. Parks http://localhost:8080/fhir/Patient/10402/_history/1
Christopher C.H. Parks http://localhost:8080/fhir/Patient/12052/_history/1
Christopher C.H. Parks http://localhost:8080/fhir/Patient/12053/_history/1
小明 陳 http://localhost:8080/fhir/Patient/12102/_history/1
大明 陳 http://localhost:8080/fhir/Patient/12202/_history/1
鴻裕 李 http://localhost:8080/fhir/Patient/9953/_history/2
大明 陳 http://localhost:8080/fhir/Patient/12103/_history/1
大明 陳 http://localhost:8080/fhir/Patient/12104/_history/1
大明 陳 http://localhost:8080/fhir/Patient/12153/_history/1
Alejandro916 Sergio619 Espinoza81 http://localhost:8080/fhir/Patient/25302/_history/1
Jamison785 Lindgren255 http://localhost:8080/fhir/Patient/697388bd-f26d-4485-8e98-c3aa204b0a87/_history/1
Sharell873 Jaskolski867 http://localhost:8080/fhir/Patie